In [2]:
import os
import cv2
import numpy as np
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw

# Define a function to convert polyline to mask
def polyline_to_mask(height, width, points):
    # Create a black image with same size as the original image
    mask = np.zeros((height, width), dtype=np.uint8)
    
    # Convert points to a numpy array
    points = np.array(points, dtype=np.int32)
    
    # Draw the polyline
    cv2.polylines(mask, [points], isClosed=False, color=1, thickness=3, lineType=cv2.FILLED)
    
    return mask

# Define a function to parse XML file and extract masks
def extract_masks_from_xml(xml_file, output_dir):
    # Parse XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()

    # Iterate over each image in the XML file
    for image in root.findall('image'):
        # Get image attributes
        image_id = image.get('id')
        image_name = image.get('name')
        width = int(image.get('width'))
        height = int(image.get('height'))

        # Iterate over each polyline in the image
        for polyline in image.findall('polyline'):
            # Get polyline points and convert them to a list of tuples
            points = polyline.get('points').split(';')
            points = [tuple(map(float, point.split(','))) for point in points]

            # Convert polyline to mask
            mask = polyline_to_mask(height, width, points)

            # Convert mask to PIL image
            mask_img = Image.fromarray(mask.astype(np.uint8) * 255)

            # Save mask to output directory
            mask_img.save(os.path.join(output_dir, f'{image_name}_mask.png'))

# Call the function to extract masks from XML file
extract_masks_from_xml('/../annotations.xml', '/../masks')
